# Tratamento e Manipulação dos dados

As etapas deste notebook têm por objetivo garantir a integridade dos dados necessários para os cálculos da receita obtida em diferentes tipos de transações realizadas pelo PicPay, além de gerar arquivos em CSV com os resultados obtidos.

As tarefas realizadas foram divididas em:
1. Importação e inspeção dos dados
2. Cálculos da tabela "transactions"
3. Cálculos da tabela "transactions_installments"
4. Criação e exportação do arquivo final

## 1. Importação e inspeção dos dados

### Importação de Biblioteca

In [96]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

### Importação dos Dados

In [97]:
transactions = pd.read_csv('transactions.csv', sep=';')

In [98]:
transactions

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate
0,1,03/01/2021,P2P,400,600,Credit card,12,1.99,2.99,3.49
1,2,14/09/2021,BILLS,650,300,Credit card,5,1.99,2.99,3.49
2,3,20/07/2021,BILLS,1200,0,Credit card,8,1.99,2.99,3.49
3,4,06/08/2021,P2P,350,800,Credit card,9,1.99,2.99,3.49
4,5,13/04/2021,P2P,3500,0,Credit card,10,1.99,2.99,3.49
5,6,24/05/2021,P2P,3420,0,Credit card,7,1.99,2.99,3.49
6,7,31/03/2021,BILLS,5000,0,Credit card,12,1.99,2.99,3.49
7,8,30/08/2021,P2P,2800,0,Credit card,12,1.99,2.99,3.49
8,9,28/08/2021,BILLS,6000,0,Credit card,4,1.99,2.99,3.49
9,10,27/09/2021,P2P,8000,0,Credit card,6,1.99,2.99,3.49


In [99]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          10 non-null     int64  
 1   transaction_date        10 non-null     object 
 2   transaction_type        10 non-null     object 
 3   transaction_value       10 non-null     int64  
 4   receiver_used_cc_limit  10 non-null     int64  
 5   payment_method          10 non-null     object 
 6   installments            10 non-null     int64  
 7   p2p_surcharge_rate      10 non-null     float64
 8   bills_surcharge_rate    10 non-null     float64
 9   installment_rate        10 non-null     float64
dtypes: float64(3), int64(4), object(3)
memory usage: 932.0+ bytes


In [100]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], dayfirst=True)

In [101]:
transactions

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49


In [102]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   transaction_id          10 non-null     int64         
 1   transaction_date        10 non-null     datetime64[ns]
 2   transaction_type        10 non-null     object        
 3   transaction_value       10 non-null     int64         
 4   receiver_used_cc_limit  10 non-null     int64         
 5   payment_method          10 non-null     object        
 6   installments            10 non-null     int64         
 7   p2p_surcharge_rate      10 non-null     float64       
 8   bills_surcharge_rate    10 non-null     float64       
 9   installment_rate        10 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(2)
memory usage: 932.0+ bytes


## 2. Cálculos da tabela "transactions"

Os cálculos realizados serão de:

| Campo                          | Descrição                                                                 |
|-------------------------------|---------------------------------------------------------------------------|
| p2p_surcharge               | Valor da taxa adicional para transações P2P                               |
| bill_surcharge             | Valor da taxa adicional para transações BILLS                             |
| surcharged_transaction_value | Valor total da transação incluindo taxas P2P e BILLS                      |
| installment_fee            | Valor total do juros a ser cobrado pelo parcelamento (valor total, não por parcela) |
| individual_installment     | Valor de cada parcela a ser paga pelo usuário que fez um pagamento parcelado com cartão de crédito |


In [103]:
transactions['p2p_surcharge'] = np.where(
    (
        (transactions['transaction_type'] == 'P2P') & 
        (transactions['transaction_value'] + transactions['receiver_used_cc_limit'] > 800) &
        (transactions['payment_method'] == 'Credit card')
    ),
    (transactions['transaction_value'] + transactions['receiver_used_cc_limit'] - 800) * transactions['p2p_surcharge_rate']/100,
    0
)

transactions['p2p_surcharge'] = np.round(transactions['p2p_surcharge'], 2)

transactions

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28


In [104]:
transactions['bills_surcharge'] = np.where(
    (
        (transactions['transaction_type'] == 'BILLS') &
        (transactions['payment_method'] == 'Credit card')
    ),
    (transactions['transaction_value'] * transactions['bills_surcharge_rate'])/100,
    0
)

transactions['bills_surcharge'] = np.round(transactions['bills_surcharge'], 2)

transactions

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge,bills_surcharge
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98,0.00
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00,19.44
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00,35.88
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96,0.00
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73,0.00
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14,0.00
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00,149.50
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80,0.00
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00,179.40
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28,0.00


In [105]:
transactions['surcharged_transaction_value'] = transactions['transaction_value'] + transactions['p2p_surcharge'] + transactions['bills_surcharge']

transactions

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge,bills_surcharge,surcharged_transaction_value
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98,0.00,403.98
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00,19.44,669.44
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00,35.88,1235.88
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96,0.00,356.96
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73,0.00,3553.73
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14,0.00,3472.14
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00,149.50,5149.50
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80,0.00,2839.80
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00,179.40,6179.40
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28,0.00,8143.28


In [106]:
s0 = transactions['surcharged_transaction_value']
j = transactions['installment_rate']/100
n = transactions['installments']

transactions['individual_installment'] = np.where(
    (
        #(transactions['transaction_type'] == 'BILLS') &
        (transactions['payment_method'] == 'Credit card') &
        (transactions['installments'] > 1) 
    ),
    (s0 * (j/(1 - (1 + j)**(-n)))),
    0
)

transactions['individual_installment'] = np.round(transactions['individual_installment'], 2)

transactions

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge,bills_surcharge,surcharged_transaction_value,individual_installment
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98,0.00,403.98,41.78
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00,19.44,669.44,148.23
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00,35.88,1235.88,179.72
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96,0.00,356.96,46.90
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73,0.00,3553.73,427.09
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14,0.00,3472.14,567.64
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00,149.50,5149.50,532.58
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80,0.00,2839.80,293.70
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00,179.40,6179.40,1681.95
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28,0.00,8143.28,1527.73


In [107]:
transactions['installment_fee'] = np.where(
    (
        #(transactions['transaction_type'] == 'BILLS') & 
        (transactions['payment_method'] == 'Credit card') &
        (transactions['installments'] > 1) 
    ),
    transactions['individual_installment'] * transactions['installments'] - transactions['surcharged_transaction_value'],
    0
)
transactions

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge,bills_surcharge,surcharged_transaction_value,individual_installment,installment_fee
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98,0.00,403.98,41.78,97.38
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00,19.44,669.44,148.23,71.71
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00,35.88,1235.88,179.72,201.88
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96,0.00,356.96,46.90,65.14
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73,0.00,3553.73,427.09,717.17
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14,0.00,3472.14,567.64,501.34
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00,149.50,5149.50,532.58,1241.46
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80,0.00,2839.80,293.70,684.60
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00,179.40,6179.40,1681.95,548.40
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28,0.00,8143.28,1527.73,1023.10


## 2. Cálculos da tabela "transactions_installments"

In [118]:
drop_columns = [
        'p2p_surcharge_rate', 
        'bills_surcharge_rate', 
        'installment_rate', 
        'p2p_surcharge', 
        'bills_surcharge', 
        'surcharged_transaction_value',
        'installments',
        'installment_fee'
        ]

rename_columns = {
    'receiver_used_cc_limit': 'used_cc_limit',
    'individual_installment': 'due_amount'
}

transactions_installments = transactions[transactions['installments'] > 1].copy()
transactions_installments['installment_number'] = transactions_installments['installments'].apply(lambda x: list(range(1, int(x) + 1)))
transactions_installments = transactions_installments.explode('installment_number')
transactions_installments = transactions_installments.reset_index(drop=True)
transactions_installments = transactions_installments.drop(drop_columns, axis='columns')
transactions_installments = transactions_installments.rename(columns=rename_columns)

transactions_installments.head(20)

,transaction_id,transaction_date,transaction_type,transaction_value,used_cc_limit,payment_method,due_amount,installment_number
0,1,2021-01-03,P2P,400,600,Credit card,41.78,1
1,1,2021-01-03,P2P,400,600,Credit card,41.78,2
2,1,2021-01-03,P2P,400,600,Credit card,41.78,3
3,1,2021-01-03,P2P,400,600,Credit card,41.78,4
4,1,2021-01-03,P2P,400,600,Credit card,41.78,5
5,1,2021-01-03,P2P,400,600,Credit card,41.78,6
6,1,2021-01-03,P2P,400,600,Credit card,41.78,7
7,1,2021-01-03,P2P,400,600,Credit card,41.78,8
8,1,2021-01-03,P2P,400,600,Credit card,41.78,9
9,1,2021-01-03,P2P,400,600,Credit card,41.78,10


In [ ]:
def calculate_installments(group):
    """
    Esta função calcula o saldo devedor, a amortização da parcela, a taxa de juros da parcela
    e o valor devido para cada parcela de uma transação, utilizando a fórmula da Tabela Price.

    Parâmetros:
    - group (DataFrame): Um grupo de parcelas para uma transação específica (agrupado por 'transaction_id').

    Retorna:
    - DataFrame: Um DataFrame contendo os valores calculados para cada parcela.
    """
    balance = group['surcharged_transaction_value'].iloc[0]  # Initial loan balance
    interest_rate = group['installment_rate'].iloc[0] / 100   # Monthly interest rate
    num_installments = group['installments'].iloc[0]  # Total number of installments

    # Fixed installment amount using the Price table formula
    installment_value = balance * (interest_rate * (1 + interest_rate) ** num_installments) / ((1 + interest_rate) ** num_installments - 1)

    new_values = []

    for i in range(num_installments):
        interest = balance * interest_rate
        amortization = installment_value - interest
        new_values.append({
            'loan_balance': balance,
            'installment_fee': interest,
            'installment_amortization': amortization,
            'due_amount': installment_value
        })
        balance -= amortization  # Reduce balance by the amortization amount

    return pd.DataFrame(new_values)

In [121]:
# Merge 'surcharged_transaction_value' and 'installment_rate' columns into the installments DataFrame
transactions_installments = transactions_installments.merge(
    transactions[['transaction_id', 'surcharged_transaction_value', 'installment_rate', 'installments']],
    on='transaction_id',
    how='left'
)

# Initialize new columns to store loan balance, installment amortization, installment fee, and due amount
transactions_installments['loan_balance'] = np.nan
transactions_installments['installment_amortization'] = np.nan
transactions_installments['installment_fee'] = np.nan
transactions_installments['due_amount'] = np.nan

# Apply the function to each transaction_id group
calculated_values = transactions_installments.groupby('transaction_id').apply(calculate_installments).reset_index(drop=True)

# Assign the calculated values to the appropriate columns
transactions_installments[['loan_balance', 'installment_fee', 'installment_amortization', 'due_amount']] = calculated_values

# Columns to round
round_columns = [
    'loan_balance', 
    'installment_fee', 
    'installment_amortization', 
    'due_amount'
]

# Round the values to 2 decimal places
transactions_installments[round_columns] = \
    transactions_installments[round_columns].apply(lambda x: np.round(x, 2))

# Drop unnecessary columns
drop_columns = [
    'surcharged_transaction_value',
    'installment_rate',
    'installments'
]

transactions_installments = transactions_installments.drop(drop_columns, axis='columns')

# Display the result
transactions_installments.head(12)

C:\Users\taina\AppData\Local\Temp\ipykernel_3100\4277634461.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  calculated_values = transactions_installments.groupby('transaction_id').apply(calculate_installments).reset_index(drop=True)


,transaction_id,transaction_date,transaction_type,transaction_value,used_cc_limit,payment_method,due_amount,installment_number,loan_balance,installment_amortization,installment_fee
0,1,2021-01-03,P2P,400,600,Credit card,41.78,1,403.98,27.68,14.10
1,1,2021-01-03,P2P,400,600,Credit card,41.78,2,376.30,28.65,13.13
2,1,2021-01-03,P2P,400,600,Credit card,41.78,3,347.65,29.65,12.13
3,1,2021-01-03,P2P,400,600,Credit card,41.78,4,318.00,30.68,11.10
4,1,2021-01-03,P2P,400,600,Credit card,41.78,5,287.32,31.75,10.03
5,1,2021-01-03,P2P,400,600,Credit card,41.78,6,255.57,32.86,8.92
6,1,2021-01-03,P2P,400,600,Credit card,41.78,7,222.70,34.01,7.77
7,1,2021-01-03,P2P,400,600,Credit card,41.78,8,188.70,35.20,6.59
8,1,2021-01-03,P2P,400,600,Credit card,41.78,9,153.50,36.42,5.36
9,1,2021-01-03,P2P,400,600,Credit card,41.78,10,117.08,37.69,4.09
